# BLAST Analysis for AIPD TEVV (Ikonomova et al.)
Returns best match for homolog sequences in DNA or amino acid FASTA files using BLAST+.

In [6]:
import time
import pandas as pd
from Bio import SeqIO, Entrez
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
from pathlib import Path
import os

In [2]:
# Set your email for NCBI
Entrez.email = "example@uni.edu"

In [3]:
def translate_dna(dna_seq):
    """Translate DNA sequence to amino acid (first frame only)"""
    return str(Seq(dna_seq).translate())

def blast_search(aa_seq):
    """Perform BLAST search and return top hit information"""
    try:
        # Search against non-redundant protein database (nr)
        result_handle = NCBIWWW.qblast("blastp",
                                       "nr",
                                       aa_seq,
                                       hitlist_size=1,
                                       alignments=1
                                       )
        blast_records = NCBIXML.parse(result_handle)
        
        # Get the first (and only) record
        record = next(blast_records)
        
        # Check if there are any alignments
        if record.alignments:
            # Get the top hit
            top_hit = record.alignments[0]
            top_hsp = top_hit.hsps[0]
            return top_hit.title, top_hsp.expect
        else:
            return "NA", "NA"
    except Exception as e:
        print(f"Error in BLAST search: {e}")
        return "NA", "NA"

def bestmatch(fasta_file, type="aa"):
    """Process fasta file and output results"""
    results = []
    
    # Parse the fasta file
    for record in SeqIO.parse(fasta_file, "fasta"):
        seq_name = record.id
        seq = str(record.seq)
        
        if type != "aa":
            # Translate DNA to amino acid
            aa_seq = translate_dna(seq)
        else:
            aa_seq = seq
        
        # Perform BLAST search
        print(f"Processing sequence: {seq_name}")
        match_name, e_value = blast_search(aa_seq)
        
        # Add to results
        results.append({
            "query_sequence": seq_name,
            "top_match": match_name,
            "e_value": e_value
        })
        
        # Sleep to avoid overwhelming the NCBI server
        # NCBI allows ~3 requests per second for registered users
        time.sleep(1)
    
    # Convert results to DataFrame
    df = pd.DataFrame(results)
    
    return df

## Process FASTA files

In [13]:
for protein in ["psd95pdz3", "ura3", "t7rnapol"]:
    fasta_file = Path(f"../data/fasta/{protein}.fasta")

    output_file = "../data/blast/" + fasta_file.stem + "_blast_results.csv"
    if not os.path.exists(output_file):
        results_df = bestmatch(fasta_file)

        # Display results
        print("\nResults:")
        print(results_df)

        # Save results to CSV
        results_df.to_csv(output_file, index=False)
        print(f"\nResults saved to {output_file}")
    else:
        print(f"{output_file} already exists. Skipping BLAST search.")

../data/blast/psd95pdz3_blast_results.csv already exists. Skipping BLAST search.
../data/blast/ura3_blast_results.csv already exists. Skipping BLAST search.
../data/blast/t7rnapol_blast_results.csv already exists. Skipping BLAST search.
